In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier


In [2]:
# load the data
temp_df=pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [3]:
df=temp_df.iloc[:10000]

In [4]:
df.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
print(f" Check for Null values ? : \n{df.isnull().sum()}\n")

print(f"Check for duplicates :" , df.duplicated().sum())

 Check for Null values ? : 
review       0
sentiment    0
dtype: int64

Check for duplicates : 17


In [6]:
# we will drop duplicates now
df.drop_duplicates(inplace=True)

# print duplicates if any
df.duplicated().sum()

/tmp/ipykernel_17/2549609413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


0

In [7]:
import re

In [8]:
def tag_remove(text):
    cleaned=re.sub("<.*?>","",text)
    
    return cleaned
    

In [9]:
# remove tags as datset has some html tags also as its scraped data
df["review"]=df["review"].apply(tag_remove)

/tmp/ipykernel_17/131045155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["review"]=df["review"].apply(tag_remove)


In [10]:
# check if dataset has tags ?

def check_tags(text):
    if re.search("<.*?>",text):
        return True

In [11]:
df["review"].apply(check_tags).sum()  
# tags removed

0

In [12]:
#  make all lowercase
df["review"]=df["review"].apply(lambda x:x.lower())

/tmp/ipykernel_17/3085173311.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["review"]=df["review"].apply(lambda x:x.lower())


In [13]:
# remove stopwords
from nltk.corpus import stopwords

stop_list=stopwords.words("english")
print(stop_list)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [14]:
def rem_stopwords(text):
    all_word=[]
    for word in text.split():
        if word not in stop_list:
            all_word.append(word)
            
            
    return " ".join(all_word)   
        

In [15]:
# stopwords removal done
df["review"]=df["review"].apply(rem_stopwords)

/tmp/ipykernel_17/3390780991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["review"]=df["review"].apply(rem_stopwords)


In [16]:
y=df["sentiment"]

In [17]:
# encode target variable with label encoder  
from sklearn.preprocessing import LabelEncoder


In [18]:
label=LabelEncoder()

y=label.fit_transform(y)

In [19]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [20]:
# now split th data into train test

from sklearn.model_selection import train_test_split

In [21]:
x=df.iloc[:,:-1]

In [22]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [23]:
# apply bow

from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()

In [24]:
X_train_bow=bow.fit_transform(X_train["review"]).toarray()
X_test_bow=bow.transform(X_test["review"]).toarray()

In [25]:
from sklearn.naive_bayes import GaussianNB

In [26]:
gnb=GaussianNB()

gnb.fit(X_train_bow,y_train)

GaussianNB()

In [27]:
y_pred=gnb.predict(X_test_bow)

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
accuracy_score(y_pred,y_test)

0.627941912869304

In [30]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred_rf = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred_rf)

0.8492739108662994

In [31]:
len(bow.vocabulary_)

48247

In [32]:
# we can decrease  max_features =3000 and seen that less the max_feature means less words from vocabulary accuracy will be good

In [33]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred_ng = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred_ng)

0.8467701552328493

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred_tf = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred_tf)

0.8542814221331998